# Data loading and preprocessing

In this notebook we retrieve our dataset and preprocess it into a format that is ready to use for training our matrix factorization based recommender system.

First, we import the necessary Python packages.

In [1]:
import gdown
import numpy as np
import pandas as pd
import scipy.sparse
from tqdm.auto import tqdm
import itertools
import json
import gzip
import math
from utils import read_json_fast
tqdm.pandas() # for progress_apply etc.

## Loading

Next, we download the files for our dataset (Goodreads). We use the gdown package to retrieve them from the Google Drive they're originally hosted on. 

> Since we will be implementing a collaborative filtering algorithm, we only need the interactions part of the dataset. The code for reading in the other parts of the dataset were left as comments for potential future reference.

We define the URLs for each file...

In [2]:
URLS = {
    # "BOOKS": "https://drive.google.com/uc?id=1ICk5x0HXvXDp5Zt54CKPh5qz1HyUIn9m",
    # "AUTHORS": "https://drive.google.com/uc?id=19cdwyXwfXx_HDIgxXaHzH0mrx8nMyLvC",
    # "REVIEWS": "https://drive.google.com/u/0/uc?id=1V4MLeoEiPQdocCbUHjR_7L9ZmxTufPFe",
    "INTERACTIONS": "https://drive.google.com/uc?id=1CCj-cQw_mJLMdvF_YYfQ7ibKA-dC_GA2"
}

and download each file. (if they haven't been downloaded in a previous run of the notebook)

In [3]:
for name, url in URLS.items():
    gdown.cached_download(url, f"./data/{name}.json.gz", quiet=False)

File exists: ./data/INTERACTIONS.json.gz


We define the dataset file locations...

In [4]:
# books_file = './data/BOOKS.json.gz' # book metadata
interactions_file = './data/INTERACTIONS.json.gz' # user-book interactions (ratings)
# reviews_file = './data/REVIEWS.json.gz' # user-book interactions (reviews)
# authors_file = './data/AUTHORS.json.gz' # author metadata

and load the necessary files.

In [5]:
%%time
# df_books = read_json_fast(books_file)
df_interactions = read_json_fast(interactions_file)
# df_authors =  read_json_fast(authors_file)

Processing INTERACTIONS.json.gz:


0lines [00:00, ?lines/s]

Wall time: 56 s


Now we look at the contents of the loaded dataset.

In [6]:
display(df_interactions.head(1))

,user_id,book_id,review_id,is_read,rating,review_text_incomplete,date_added,date_updated,read_at,started_at
0,8842281e1d1347389f2ab93d60773d4d,836610,6b4db26aafeaf0da77c7de6214331e1e,False,0,,Mon Aug 21 12:11:00 -0700 2017,Mon Aug 21 12:11:00 -0700 2017,,


We can see that the dataset contains quite a few columns that are of no use to us. To make everything a little less cluttered we remove the columns that we don't use from the dataframe.

In [7]:
df_interactions = df_interactions[['user_id', 'book_id', 'rating']]

In [8]:
display(df_interactions.head(1))

,user_id,book_id,rating
0,8842281e1d1347389f2ab93d60773d4d,836610,0


## Pre-processing

Now we define a pre-processing function that removes users that occur in less than minsup interactions.

In [33]:
def preprocess(df, minsup_user=5, minsup_item=35):
    """
    Goal: - Remove users that have less than minsup interactions
               
    :input df: Dataframe containing user_id and item_id 
    """
    # drop items with less then minsup user interactions
    before = df.shape[0]
    g1 = df.groupby('book_id', as_index=False)['user_id'].size()
    g2 = df.groupby('user_id', as_index=False)['book_id'].size()
    g1 = g1.rename({'size': 'user_sup'}, axis='columns')
    g1 = g1[g1.user_sup >= minsup_item]
    df = pd.merge(df, g1, how='inner', on=['book_id'])
    print(f"After dropping items with less than {minsup_item} interactions: {before} -> {df.shape[0]}")
    # drop users with less then minsup items in history
    before = df.shape[0]
    g2 = g2.rename({'size': 'user_sup'}, axis='columns')
    g2 = g2[g2.user_sup >= minsup_user]
    df = pd.merge(df, g2, how='inner', on=['user_id'])
    print(f"After dropping users with less than {minsup_user} interactions: {before} -> {df.shape[0]}")
    return df

Then we apply the pre-processing function to the dataframe and log the change in number of samples, number of unique users and number of unique items.

In [34]:
#print number of users and items
print(f"number of unique users: {df_interactions['user_id'].nunique()}")
print(f"number of unique items: {df_interactions['book_id'].nunique()}")
processed_df_interactions = preprocess(df_interactions.copy())
# display(processed_df_interactions.head(5))
print(f"number of unique users: {processed_df_interactions['user_id'].nunique()}")
print(f"number of unique items: {processed_df_interactions['book_id'].nunique()}")
# create sequential ids
processed_df_interactions['user_id_seq'] = processed_df_interactions['user_id'].astype('category').cat.codes
processed_df_interactions['book_id_seq'] = processed_df_interactions['book_id'].astype('category').cat.codes
# merge book id and rating for easier 
display(processed_df_interactions.head(5))

number of unique users: 342415
number of unique items: 89411
After dropping items with less than 35 interactions: 7347630 -> 6686728
After dropping users with less than 5 interactions: 6686728 -> 6355864
number of unique users: 148304
number of unique items: 26431


,user_id,book_id,rating,user_sup_x,user_sup_y,user_id_seq,book_id_seq
0,8842281e1d1347389f2ab93d60773d4d,836610,0,2227,14,79025,24838
1,8842281e1d1347389f2ab93d60773d4d,7648967,0,218,14,79025,23911
2,8842281e1d1347389f2ab93d60773d4d,15704307,0,26157,14,79025,4944
3,8842281e1d1347389f2ab93d60773d4d,6902644,0,1356,14,79025,22859
4,8842281e1d1347389f2ab93d60773d4d,9844623,0,1163,14,79025,26319


We store the mapping from user/book id's to their sequential id in an external file. This might come in handy in other notebooks.

In [35]:
processed_df_interactions[['user_id', 'user_id_seq']].drop_duplicates().to_pickle("./group/user_id_map_group.pkl")
processed_df_interactions[['book_id', 'book_id_seq']].drop_duplicates().to_pickle("./group/book_id_map_group.pkl")

We group the interactions by user.

In [71]:
# Group per user
grouped_df = processed_df_interactions.groupby(by='user_id_seq')[['book_id_seq', 'rating']].agg(list)

In [72]:
grouped_df.head()

,book_id_seq,rating
user_id_seq,,
0,"[14718, 20012, 15687, 15681, 15690, 15695, 156...","[4, 5, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, ..."
1,"[19644, 13083, 13078, 20743, 10169, 19405, 183...","[4, 4, 0, 0, 0, 3, 4, 3, 0, 0, 3, 3, 3, 4]"
2,"[21787, 3072, 3736, 6849, 24072, 11054, 9804, ...","[0, 4, 4, 5, 5, 4, 4, 0, 0, 4, 4, 4, 5, 4, 5, ..."
3,"[25919, 4788, 13180, 25656, 25930, 25932, 1958...","[5, 4, 5, 3, 4, 3, 4, 4, 4, 4, 4, 4, 4, 3, 4, ..."
4,"[9609, 4043, 7052]","[2, 4, 0]"


We zip the book_id_seq and rating columns for easier random splitting.

In [73]:
grouped_df['interactions'] = grouped_df.progress_apply(lambda row: list(zip(row['book_id_seq'], row['rating'])), axis=1)
grouped_df = grouped_df[grouped_df['interactions'].map(len) > 1]
grouped_df.head()

  0%|          | 0/148304 [00:00<?, ?it/s]

,book_id_seq,rating,interactions
user_id_seq,,,
0,"[14718, 20012, 15687, 15681, 15690, 15695, 156...","[4, 5, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[(14718, 4), (20012, 5), (15687, 4), (15681, 4..."
1,"[19644, 13083, 13078, 20743, 10169, 19405, 183...","[4, 4, 0, 0, 0, 3, 4, 3, 0, 0, 3, 3, 3, 4]","[(19644, 4), (13083, 4), (13078, 0), (20743, 0..."
2,"[21787, 3072, 3736, 6849, 24072, 11054, 9804, ...","[0, 4, 4, 5, 5, 4, 4, 0, 0, 4, 4, 4, 5, 4, 5, ...","[(21787, 0), (3072, 4), (3736, 4), (6849, 5), ..."
3,"[25919, 4788, 13180, 25656, 25930, 25932, 1958...","[5, 4, 5, 3, 4, 3, 4, 4, 4, 4, 4, 4, 4, 3, 4, ...","[(25919, 5), (4788, 4), (13180, 5), (25656, 3)..."
4,"[9609, 4043, 7052]","[2, 4, 0]","[(9609, 2), (4043, 4), (7052, 0)]"


We create 5 random 80/20 train/test splits for validation.

In [74]:
from sklearn.model_selection import train_test_split as split

In [75]:
# Split dataset into 0.8 training and 0.2 test samples, split randomly into 5 folds
percentage_train = 0.8

for i in range(1,6):
    grouped_df[[f'train{i}', f'test{i}']] = grouped_df.progress_apply(
        lambda row: split(row['interactions'], train_size=percentage_train, random_state=i), 
        axis=1, 
        result_type='expand'
    )

  0%|          | 0/148001 [00:00<?, ?it/s]

  0%|          | 0/148001 [00:00<?, ?it/s]

  0%|          | 0/148001 [00:00<?, ?it/s]

  0%|          | 0/148001 [00:00<?, ?it/s]

  0%|          | 0/148001 [00:00<?, ?it/s]

In [76]:
grouped_df.head()

,book_id_seq,rating,interactions,train1,test1,train2,test2,train3,test3,train4,test4,train5,test5
user_id_seq,,,,,,,,,,,,,
0,"[14718, 20012, 15687, 15681, 15690, 15695, 156...","[4, 5, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[(14718, 4), (20012, 5), (15687, 4), (15681, 4...","[(15691, 4), (15682, 4), (15690, 4), (15687, 4...","[(15680, 4), (20013, 5), (15681, 4), (15686, 4...","[(15684, 4), (15681, 4), (15679, 4), (15688, 4...","[(15676, 5), (14718, 4), (15678, 4), (15689, 4...","[(15684, 4), (20012, 5), (15687, 4), (15676, 5...","[(15678, 4), (15679, 4), (15691, 4), (15692, 4...","[(15682, 4), (15689, 4), (15694, 4), (15687, 4...","[(15678, 4), (15676, 5), (14718, 4), (15681, 4...","[(19879, 4), (15680, 4), (15682, 4), (15695, 3...","[(15683, 4), (15687, 4), (15691, 4), (20013, 5..."
1,"[19644, 13083, 13078, 20743, 10169, 19405, 183...","[4, 4, 0, 0, 0, 3, 4, 3, 0, 0, 3, 3, 3, 4]","[(19644, 4), (13083, 4), (13078, 0), (20743, 0...","[(13078, 0), (7657, 3), (10169, 0), (13083, 4)...","[(20743, 0), (16656, 3), (18302, 4)]","[(19644, 4), (2618, 0), (20743, 0), (13083, 4)...","[(21056, 3), (10169, 0), (19405, 3)]","[(13078, 0), (24036, 4), (18302, 4), (19405, 3...","[(16656, 3), (10169, 0), (13083, 4)]","[(2618, 0), (18302, 4), (24036, 4), (13078, 0)...","[(10169, 0), (20743, 0), (21056, 3)]","[(13078, 0), (7657, 3), (24036, 4), (21056, 3)...","[(19405, 3), (13083, 4), (16656, 3)]"
2,"[21787, 3072, 3736, 6849, 24072, 11054, 9804, ...","[0, 4, 4, 5, 5, 4, 4, 0, 0, 4, 4, 4, 5, 4, 5, ...","[(21787, 0), (3072, 4), (3736, 4), (6849, 5), ...","[(18898, 5), (13069, 0), (16479, 5), (24072, 5...","[(9804, 4), (6849, 5), (5238, 4), (3736, 4)]","[(3798, 4), (11054, 4), (6849, 5), (3072, 4), ...","[(5191, 4), (24072, 5), (3369, 0), (21787, 0)]","[(24072, 5), (5238, 4), (9804, 4), (13069, 0),...","[(4058, 4), (3736, 4), (3072, 4), (18898, 5)]","[(24072, 5), (21787, 0), (16479, 5), (19299, 5...","[(9804, 4), (6849, 5), (3369, 0), (4058, 4)]","[(4058, 4), (1800, 0), (19299, 5), (13069, 0),...","[(11054, 4), (3072, 4), (3798, 4), (3736, 4)]"
3,"[25919, 4788, 13180, 25656, 25930, 25932, 1958...","[5, 4, 5, 3, 4, 3, 4, 4, 4, 4, 4, 4, 4, 3, 4, ...","[(25919, 5), (4788, 4), (13180, 5), (25656, 3)...","[(13180, 5), (1554, 4), (25930, 4), (19897, 4)...","[(25656, 3), (446, 3), (19589, 4), (21853, 4)]","[(17940, 4), (25932, 3), (25656, 3), (21853, 4...","[(26152, 4), (25930, 4), (20244, 3), (25919, 5)]","[(25930, 4), (446, 3), (19589, 4), (25921, 4),...","[(1554, 4), (13180, 5), (4788, 4), (19897, 4)]","[(25926, 3), (25930, 4), (25919, 5), (19897, 4...","[(5952, 4), (25656, 3), (20244, 3), (19589, 4)]","[(26152, 4), (4788, 4), (24897, 4), (21853, 4)...","[(13180, 5), (25932, 3), (19897, 4), (5952, 4)]"
4,"[9609, 4043, 7052]","[2, 4, 0]","[(9609, 2), (4043, 4), (7052, 0)]","[(7052, 0), (4043, 4)]","[(9609, 2)]","[(4043, 4), (9609, 2)]","[(7052, 0)]","[(9609, 2), (7052, 0)]","[(4043, 4)]","[(9609, 2), (7052, 0)]","[(4043, 4)]","[(4043, 4), (7052, 0)]","[(9609, 2)]"


Finally, we create a sparse representation of the user-item interaction matrix for our train and test set.

In [77]:
def create_sparse_repr(df, column, shape):
    user_ids = []
    item_ids = []
    values = []
    for idx, row in tqdm(df.iterrows()):
        interactions = row[column]
        items, ratings = list(zip(*interactions))
        item_ids.extend(items)
        values.extend(ratings)
        user_ids.extend([idx] * len(items))
    matrix = scipy.sparse.coo_matrix((values, (user_ids, item_ids)), shape=shape, dtype=np.int32)
    return matrix
    

shape = (processed_df_interactions['user_id_seq'].max() + 1,  processed_df_interactions['book_id_seq'].max() + 1)

for i in range(1,6):
    train = create_sparse_repr(grouped_df, column=f'train{i}', shape=shape)
    test = create_sparse_repr(grouped_df, column=f'test{i}', shape=shape)
    # We store the train and test set externally to be used in the training and evaluating notebook.
    scipy.sparse.save_npz(f'./group/train{i}.npz', train)
    scipy.sparse.save_npz(f'./group/test{i}.npz', test)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]